## Microtubule Dynamics ABM Julia notebook
This notebook is intended as a development and testing file , so i will change regulary.

nächstes meeting 05.10 10:45

### _TODO_ ::

    - Drei Abbildungen erstellen für klein 100, mittel300, groß1000 ,a 50 mal begrenzte randbedingungen ✓
    - seed funktion?!
    - Standartdabweichung (korridor) 
    - Rohdaten gut aufbewahren, sinnvolle beschriftungen  ✓
    - Periodischen Randbedingugen mit in microtubuli modell implementieren ✓

  - Wachstumsgeswindigkeit (erstmal ohne Depolymerisation) ~ N.Agenten/Gridsize & P-polymerisiert


In [1]:
using Agents
using CairoMakie
using Distributions
using DataFrames
using InteractiveDynamics
using GLMakie
#import DrWatson: @dict

In [2]:
#=  Agent class representing 1 tubulin dimer
    those can polymerize and also hydrolize from GTP to GDP
maybe get a plus end direction in near future =#

@agent tubulin GridAgent{2} begin
    polym::Bool
    GDP::Bool
    # Plus::Int64
end

In [3]:
#= Main Function to initialize the model=#

function initialize(;Nstarts=1, numagents = 550, periodic=false,
        griddims = (100,100), 
        p_hyd= 0.001, 
        p_polym = 0.99,
        p_depolym_GTP= 0.005,
        p_depolym_GDP=0.01 )
   
    # Probability to hydrolize from GTP to GDP: 
    P_hyd = Binomial(1,p_hyd)
    # Probability to Polymerize/bind to the microtuble 
    P_polym = Binomial(1,p_polym)
    # Probability to depolimerize if not hydrolized (GTP - tubulin) = lower
    P_depolym_GTP = Binomial(1,1-p_depolym_GTP)
    # Probability to depolimerize if hydrolized (GDP - tubulin) = higher
    P_depolym_GDP = Binomial(1,1-p_depolym_GDP)
    
    
    properties = Dict(:griddims=> griddims,
                    :periodic=>periodic,
                    :numagents => numagents,
                    :Nstarts => Nstarts,
                    :p_hyd => p_hyd,
                    :p_polym => p_polym,
                    :p_depolym_GTP => p_depolym_GTP,
                    :p_depolym_GDP => p_depolym_GDP,
                    :P_hyd => P_hyd,
                    :P_polym=> P_polym,
                    :P_depolym_GTP=>  P_depolym_GTP,
                    :P_depolym_GDP=> P_depolym_GDP
                    )
    
    
    space = GridSpace(griddims, periodic= periodic)
    
    model = ABM(
        tubulin,space;
        scheduler = Schedulers.randomly,
        properties = properties )
    
    # populize Space
    id = 0
    for _ in 1:Nstarts # Startingpoints:
        id += 1
        agent =  tubulin(id,(50,50),true,true)
        if id == 1
            add_agent_pos!(agent,model)
        else
            add_agent_single!(agent,model) # add additional startingpoints to random positions 
        end
    end
    for _ in 1:numagents 
        id += 1
        agent =  tubulin(id,(1,1),false,false)
        add_agent_single!(agent,model) # add agents to random positions
    end
    return model
end

initialize (generic function with 1 method)

In [3]:
# Agent step func 
function agent_step!(agent,model)
    # Starting points have to stay the same
    if agent.id > model.Nstarts  
        # Logic for unpolymerized agents:
        if agent.polym == false            
            walk!(agent, rand, model, ifempty = true)# brownsche bewegung wenn nicht polymerisiert
            for id in nearby_ids(agent, model, [(1, 1:1), (2, 1:1)]) # check topright neighbour
                if model[id].polym # if polymerized
                    agent.polym = rand(model.P_polym,1)[1] # polimerize with P_polym
                end
            end
        end
        # Logic for polymierized agents:
        if agent.polym
            if agent.GDP == false   # if not hydrolized
                agent.GDP = rand(model.P_hyd,1)[1] # hydrolize with P_hyd
            end
            if isempty(tuple(agent.pos[1]-1,agent.pos[2]-1),model) # check if end of microtuble
                if agent.GDP  # if hydrolized
                        agent.polym = rand(model.P_depolym_GDP,1)[1] # depolyimerize with P_depolym_GDP
                    else
                        agent.polym = rand(model.P_depolym_GTP,1)[1] # depolymerize with P_depolym_GDP     
                end
            end
        end
    end 
end

agent_step! (generic function with 1 method)

In [4]:
# Periodic agent step function
 # TODO: bounce check für agenten in eigene funktion aussiedeln
function agent_step!(agent,model)
    if agent.id > model.Nstarts  
        if agent.polym == false   # Logic for unpolymerized agents:    
             walk!(agent, rand, model, ifempty =true)#
            for id in nearby_ids(agent, model,1) # 
                if  model[id].polym == true
                   if model[id].pos[1] == agent.pos[1] + 1 && model[id].pos[2] == agent.pos[2] + 1  || model[id].pos[1] == 1 && agent.pos[1] == model.griddims[1] && model[id].pos[2] == agent.pos[2]+1 || model[id].pos[1] == agent.pos[1] + 1 && model[id].pos[2] == 1 && agent.pos[2] == model.griddims[2]|| model[id].pos[1] == 1 && agent.pos[1] == model.griddims[1] && model[id].pos[2] == 1 && agent.pos[2] == model.griddims[2]
                        agent.polym = rand(model.P_polym,1)[1] 
                    end
                end
             end
        end
    if agent.polym
        if agent.GDP == false   # if not hydrolized
            agent.GDP = rand(model.P_hyd,1)[1] # hydrolize with P_hyd
        end
         if agent.pos[1]>1 && agent.pos[2]>1 && isempty(tuple(agent.pos[1]-1,agent.pos[2]-1),model) || agent.pos[1]==1 && agent.pos[2]>1 && isempty(tuple(model.griddims[1],agent.pos[2]-1),model)   || agent.pos[1]>1 && agent.pos[2]==1 && isempty(tuple(agent.pos[1]-1,model.griddims[2]),model)|| agent.pos[1]==1 && agent.pos[2]==1 && isempty(tuple(model.griddims[2],model.griddims[2]),model) || agent.pos[1] == model.griddims[1] && agent.pos[2]>1 && isempty(tuple(1, agent.pos[2]-1))|| agent.pos[1]>1 && agent.pos[2] == model.griddims[2] && isempty(tuple(agent.pos[2]-1,1 ))     
                if agent.GDP  # if hydrolized
                    agent.polym = rand(model.P_depolym_GDP,1)[1] # depolyimerize with P_depolym_GDP
                else
                    agent.polym = rand(model.P_depolym_GTP,1)[1]
                end
            end 
        end
    end
end

agent_step! (generic function with 1 method)

In [5]:
# model-Step func:
function model_step!(model)
    # calculate mean microtuble length 
    

end

model_step! (generic function with 1 method)

In [16]:
# initialization
model = initialize(
    griddims =(200,200),
    periodic= true,
    Nstarts=15,
    numagents = 1250,
    p_polym=0.80,
    p_hyd= 0.02,
    p_depolym_GTP=0,
    p_depolym_GDP=0.03)


AgentBasedModel with 1265 agents of type tubulin
 space: GridSpace with size (200, 200), metric=chebyshev, periodic=true
 scheduler: randomly
 properties: Dict{Symbol, Any}(:griddims => (200, 200), :P_hyd => Binomial{Float64}(n=1, p=0.02), :p_depolym_GDP => 0.03, :Nstarts => 15, :P_depolym_GTP => Binomial{Float64}(n=1, p=1.0), :periodic => true, :P_depolym_GDP => Binomial{Float64}(n=1, p=0.97), :p_polym => 0.8, :p_hyd => 0.02, :p_depolym_GTP => 0…)

In [17]:
#Video out:

# coloring

groupcolor(a) = a.polym == true ? :red : :blue
groupmarker(a) = a.GDP == true ? :circle : '○'

#GLMakie.activate!()

abm_video(
    "Microtubule_periodic_15starts.mp4", model, agent_step!,model_step!;
    ac = groupcolor, am = groupmarker,
    framerate = 20, frames = 8500,
    title = "Microtubule Simulation Startingblock: ggriddims =(200,200),
    periodic= true,
    Nstarts=10,
    numagents = 1250,
    p_polym=0.80,
    p_hyd= 0.03,
    p_depolym_GTP=0,
    p_depolym_GDP=0.02")


In [6]:
### Data collection

adata = [:pos, :polym, :GDP]

model = initialize(griddims =(80,80),
    Nstarts=0,
    numagents = 1000,
    p_polym=0.99,
    p_hyd= 0.1,
    p_depolym_GTP=0.02,
    p_depolym_GDP=0.04)

data, _ = run!(model,agent_step!, model_step!,600;adata)




(601000×5 DataFrame
    Row │ step   id     pos       polym  GDP   
        │ Int64  Int64  Tuple…    Bool   Bool  
────────┼──────────────────────────────────────
      1 │     0      1  (22, 1)   false  false
      2 │     0      2  (49, 28)  false  false
      3 │     0      3  (60, 53)  false  false
      4 │     0      4  (63, 42)  false  false
      5 │     0      5  (79, 30)  false  false
      6 │     0      6  (16, 34)  false  false
      7 │     0      7  (14, 73)  false  false
      8 │     0      8  (6, 9)    false  false
      9 │     0      9  (38, 71)  false  false
     10 │     0     10  (36, 78)  false  false
     11 │     0     11  (53, 12)  false  false
   ⋮    │   ⋮      ⋮       ⋮        ⋮      ⋮
 600991 │   600    991  (37, 56)  false  false
 600992 │   600    992  (45, 10)  false  false
 600993 │   600    993  (78, 33)  false  false
 600994 │   600    994  (21, 73)  false  false
 600995 │   600    995  (21, 29)  false  false
 600996 │   600    996  (4, 40)   false

In [10]:
# MSD function
function MSD(data)   
    n_ids= maximum(data.id)
    n_steps=maximum(data.step)+1
    r=1:n_ids:n_steps*n_ids+n_ids
    poses=hcat(collect.(data.pos)...)
    x_0=poses[1,unique(data.id)]
    y_0=poses[2,unique(data.id)]
    MSD = zeros(n_steps) #array länge data.step init zeros()
        for i in 1:n_steps
            MSD[i]=mean(sqrt.((poses[1,r[i]:r[i+1]-1].-x_0).^2 + (poses[2,r[i]:r[i+1]-1].-y_0).^2))
        end
    return MSD
end

MSD (generic function with 1 method)

In [7]:
## Ensemble run RW
models = [initialize(; Nstarts=0,
        periodic= true,
        numagents = 1000,
        griddims=(x,x), 
        p_hyd=0,
        p_polym=0,
        p_depolym_GTP=0,
        p_depolym_GDP=0) for x in repeat([80, 100, 200], 30)];

adata = [:pos]

adf, = ensemblerun!(models, agent_step!, model_step!, 2000; adata)
using CSV
CSV.write("Periodic_ensemble_80_100_200_1kagents_a30_2500_step_runs.csv", adf)

(180090000×4 DataFrame
       Row │ step   id     pos         ensemble 
           │ Int64  Int64  Tuple…      Int64    
───────────┼────────────────────────────────────
         1 │     0      1  (77, 25)           1
         2 │     0      2  (5, 51)            1
         3 │     0      3  (44, 59)           1
         4 │     0      4  (13, 64)           1
         5 │     0      5  (29, 49)           1
         6 │     0      6  (4, 66)            1
         7 │     0      7  (11, 13)           1
         8 │     0      8  (3, 45)            1
         9 │     0      9  (26, 21)           1
        10 │     0     10  (10, 48)           1
        11 │     0     11  (5, 40)            1
     ⋮     │   ⋮      ⋮        ⋮          ⋮
 180089991 │  2000    991  (6, 200)          90
 180089992 │  2000    992  (189, 22)         90
 180089993 │  2000    993  (183, 179)        90
 180089994 │  2000    994  (177, 77)         90
 180089995 │  2000    995  (47, 28)          90
 180089996 │  2000

In [18]:
## Ensemble run Microtubuli simulation
models = [initialize(; Nstarts=15,
        periodic= true,
        numagents = 1000,
        griddims=(x,x), 
        p_polym=0.80,
        p_hyd= 0.02,
        p_depolym_GTP=0,
        p_depolym_GDP=0.03) for x in repeat([120,150, 200], 30)];

adata = [:pos]

adf, = ensemblerun!(models, agent_step!, model_step!, 2000; adata)

(182791350×4 DataFrame
       Row │ step   id     pos         ensemble 
           │ Int64  Int64  Tuple…      Int64    
───────────┼────────────────────────────────────
         1 │     0      1  (50, 50)           1
         2 │     0      2  (3, 79)            1
         3 │     0      3  (82, 111)          1
         4 │     0      4  (50, 120)          1
         5 │     0      5  (33, 31)           1
         6 │     0      6  (96, 59)           1
         7 │     0      7  (55, 87)           1
         8 │     0      8  (89, 72)           1
         9 │     0      9  (9, 60)            1
        10 │     0     10  (92, 65)           1
        11 │     0     11  (2, 76)            1
     ⋮     │   ⋮      ⋮        ⋮          ⋮
 182791341 │  2000   1006  (26, 168)         90
 182791342 │  2000   1007  (124, 124)        90
 182791343 │  2000   1008  (5, 5)            90
 182791344 │  2000   1009  (111, 60)         90
 182791345 │  2000   1010  (177, 134)        90
 182791346 │  2000

In [20]:
## MSD for multiple ensembles

MSDS = DataFrame(Step= levels(adf.step))
for i in levels(adf.ensemble)
    name = Symbol("ens_$i")
    MSDS[!,name] = MSD(adf[(adf.ensemble .== i ),:])
end

CSV.write("Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv", MSDS)

"Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv"

In [19]:
CSV.write("Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv", adf)

"Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv"